In [1]:
using Pkg; Pkg.activate(".")
using CSV
using DataFrames
using PyPlot
using LaTeXStrings
using MUST
using DelimitedFiles
using Statistics
MUST.@import_dispatch "../../../dispatch2"

  Activating environment at `/raven/ptmp/peitner/model_grids/MUST.jl/examples/running_dispatch/Project.toml`


PyObject <module 'dispatch' from '/raven/ptmp/peitner/model_grids/dispatch2/utilities/python/dispatch/__init__.py'>

In [2]:
results = DataFrame(CSV.File("summary_testrun1.csv"));

In [3]:
t, rho, ee0, ee_min, successP1, successP2 = eachcol(results[:,[:stellar_params!tt_k, :stellar_params!d_cgs, :newton_params!ee0, :stellar_params!ee_min, :phase1_success, :phase2_success]]);

# Phase 1 (no RT)

In [9]:
plt.close()
im = plt.scatter(t, rho, c=successP1, cmap="bwr_r")
plt.xlabel(L"\rm T_{ini}", fontsize="x-large")
plt.ylabel(L"\rm \rho_{ini}", fontsize="x-large")
cbar = plt.colorbar(im)
cbar.set_label("phase 1 (no RT)")
cbar.set_ticks([0,1])
cbar.set_ticklabels(["failed", "success"])
plt.savefig("t_rho_p1_success.png")

In [10]:
plt.close()
im = plt.scatter(ee0, ee_min, c=successP1, cmap="bwr_r")
plt.xlabel(L"\rm ee_{0}", fontsize="x-large")
plt.ylabel(L"\rm ee_{min}", fontsize="x-large")
cbar = plt.colorbar(im)
cbar.set_label("phase 1 (no RT)")
cbar.set_ticks([0,1])
cbar.set_ticklabels(["failed", "success"])
cbar.set_ticklabels(["failed", "success"])
plt.savefig("ee0_eemin_p1_success.png")

# Phase 2 (with RT)

In [11]:
plt.close()
im = plt.scatter(t, rho, c=successP2, cmap="bwr_r")
plt.xlabel(L"\rm T_{ini}", fontsize="x-large")
plt.ylabel(L"\rm \rho_{ini}", fontsize="x-large")
cbar = plt.colorbar(im)
cbar.set_label("phase 2 (no RT)")
cbar.set_ticks([0,1])
cbar.set_ticklabels(["failed", "success"])
plt.savefig("t_rho_p2_success.png")

In [17]:
plt.close()
im = plt.scatter(ee0, ee_min, c=successP2, cmap="bwr_r")
plt.xlabel(L"\rm ee_{0}", fontsize="x-large")
plt.ylabel(L"\rm ee_{min}", fontsize="x-large")
cbar = plt.colorbar(im)
cbar.set_label("phase 2 (no RT)")
cbar.set_ticks([0,1])
cbar.set_ticklabels(["failed", "success"])
plt.savefig("ee0_eemin_p2_success.png")

# Effective temperature of successful runs

In [12]:
get_teff(folder) = begin
    path = joinpath(folder, "teff.dat")
    if !ispath(path) @warn "$(path) does not exist."
        return nothing
    end
    data = readdlm(path)
end 

get_teff (generic function with 1 method)

In [13]:
folders = [@MUST.in_dispatch(joinpath("data",r[1:first(findfirst(".nml",r))-1])) for r in results[results.phase2_success .== true, :phase2_name]];

In [14]:
teff = [get_teff(f) for f in folders];

┌ Warning: /raven/ptmp/peitner/model_grids/dispatch2/experiments/stellar_atmospheres/data/grid4_phase2/teff.dat does not exist.
└ @ Main /ptmp/peitner/model_grids/MUST.jl/examples/running_dispatch/Valid_T_rho_comb.ipynb:3
┌ Warning: /raven/ptmp/peitner/model_grids/dispatch2/experiments/stellar_atmospheres/data/grid19_phase2/teff.dat does not exist.
└ @ Main /ptmp/peitner/model_grids/MUST.jl/examples/running_dispatch/Valid_T_rho_comb.ipynb:3


┌ Warning: /raven/ptmp/peitner/model_grids/dispatch2/experiments/stellar_atmospheres/data/grid32_phase2/teff.dat does not exist.
└ @ Main /ptmp/peitner/model_grids/MUST.jl/examples/running_dispatch/Valid_T_rho_comb.ipynb:3
┌ Warning: /raven/ptmp/peitner/model_grids/dispatch2/experiments/stellar_atmospheres/data/grid51_phase2/teff.dat does not exist.
└ @ Main /ptmp/peitner/model_grids/MUST.jl/examples/running_dispatch/Valid_T_rho_comb.ipynb:3


In [16]:
plt.close()
for r in teff
    isnothing(r) && continue
    
    plt.plot(r[:,1], r[:,2], marker=".")
end
plt.ylim(4500,5750)
plt.savefig("teff.png")

In [18]:
teff_t = []
for t in teff
    try
        isnothing(t) ? append!(teff_t, [NaN]) : append!(teff_t, [mean(t[end-20:end,2])])
    catch
        append!(teff_t, [NaN])
    end
end

In [19]:
t_success, rho_success = eachcol(results[results.phase2_success .== true, [:stellar_params!tt_k, :stellar_params!d_cgs]]);

In [20]:
plt.close()
im = plt.scatter(t_success, rho_success, c=teff_t, cmap="gnuplot", s=50)
cbar = plt.colorbar(im)
plt.xlabel(L"\rm T_{ini}", fontsize="x-large")
plt.ylabel(L"\rm \rho_{ini}", fontsize="x-large")
cbar.set_label(L"\rm T_{eff}", fontsize="x-large")
plt.savefig("t_rho_teff.png")